Generate IIIF `manifest.json` (version 2)
===

In [ ]:
import os
import json
import requests

from jinja2 import Template

### fetch image URLs from a JSON file

In [ ]:
urls=[]
json_file = '../src/main/webapp/data/data-remote.json'
with open(json_file, 'r') as fp:
    for item in json.load(fp)['items']:
        for url in item['item']:
            urls.append(url)

In [ ]:
#urls

### fetch deposit metadata in JSON via REST API

In [ ]:
url = 'https://trng-repository.surfsara.nl/api/objects/deposit/900c341c1c10fff7'
title = ''
description = ''
license = ''
creator = ''
with requests.get(url) as req:
    data = json.loads(req.text)['result']['metadata']['base']
    title = data['title']
    description = data['description']
    license = data['rights'][0]['url']
    creator = data['creator'][0]

In [ ]:
#data

In [ ]:
cmd = 'git rev-parse HEAD'
git_sha = os.popen(cmd).read().strip()

### serialize `manifest.json`

In [ ]:
template = """
{
  {%- set base = urls[0] %}
  {%- set book_id = ':'.join(base.split(':')[:3]) %}
  {%- set seq_id = book_id + '/sequence/default' %}
  {#- set canvas & thumbnail dimensions #}
  {%- set cvs_dim = dict(w=1024, h=768) %}
  {%- set thb_dim = dict(w=120, h=120) %}
  "@context": "http://iiif.io/api/presentation/2/context.json",
  "@id": "https://raw.githubusercontent.com/LINNAE-project/SFB-Annotator/{{ git_sha }}/data/manifest.json",
  "@type": "sc:Manifest",
  "label": "{{ title }}",
  "metadata": [
    {"label": "Author", "value": "{{ creator }}"}
  ],
  "description": "{{ description }}",
  "thumbnail": {
    "@id": "{{ book_id + ':MMNAT01_PM_NNM001001033_001/full/!120,120/0/default.jpg' }}",
    "@type": "dctypes:Image",
    "format": "image/jpeg",
    "width": {{ thb_dim['h'] }},
    "height": {{ thb_dim['w'] }},
    "service": {
      "@context": "http://iiif.io/api/image/2/context.json",
      "@id": "http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033",
      "profile": "http://iiif.io/api/image/2/level2.json",
      "protocol": "http://iiif.io/api/image"
    }
  },
  "viewingDirection": "left-to-right",
  "viewingHint": "paged",
  "license": "{{ license }}",
  "attribution": "{{ creator }}",
  "sequences": [
    {
      "@id": "{{ book_id }}/sequence/default",
      "@type": "sc:Sequence",
      "label": "Default",
      "startCanvas": "{{ book_id }}/canvas/p1",
      "canvases": [
      {%- for url in urls %}
        {
          {%- set page = url.split('/')[-5].split('_')[-1] | int() %}
          {%- set img_id = '/'.join(url.split('/')[:-4]) %}
          {#- some fixes required in URLs #}
          {%- set cvs_id = book_id + '/canvas/p' + page|string() %}
          {%- set thb_id = img_id + '/full/!120,120/0/default.jpg' %}
          {%- set res_id = img_id + '/full/max/0/default.jpg' %}
          {%- set ano_id = img_id + '/annotation/p' + page|string() %}
          {%- set srv_id = img_id %}
          "@id": "{{ cvs_id }}",
          "@type": "sc:Canvas",
          "label": "p. {{ page }}",
          "width": {{ cvs_dim['w'] }},
          "height": {{ cvs_dim['h'] }},
          "thumbnail": {
            "@id" : "{{ thb_id }}",
            "@type": "dctypes:Image",
            "format": "image/jpeg",
            "width": {{ thb_dim['h'] }},
            "height": {{ thb_dim['w'] }},
            "service": {
              "@context": "http://iiif.io/api/image/2/context.json",
              "@id": "{{ img_id }}",
              "profile": "http://iiif.io/api/image/2/level2.json",
              "protocol": "http://iiif.io/api/image"
            }
          },
          "images": [
            {
              "@id": "{{ ano_id }}",
              "@type": "oa:Annotation",
              "motivation": "sc:painting",
              "on": "{{ cvs_id }}",
              "resource": {
                "@id": "{{ res_id }}",
                "@type": "dctypes:Image",
                "format": "image/jpeg",
                "service": {
                  "@context": "http://iiif.io/api/image/2/context.json",
                  "@id": "{{ srv_id }}",
                  "profile": "http://iiif.io/api/image/2/level2.json",
                  "protocol": "http://iiif.io/api/image"
                }
              }
            }
          ]
        }
        {{- "," if not loop.last -}}
      {% endfor %}
      ]
    }
  ]
}
"""

j2_template = Template(template)

In [ ]:
content = j2_template.render(git_sha=git_sha,
                             title=title,
                             description=description,
                             license=license,
                             creator=creator,
                             urls=urls)

In [ ]:
#print(content)

In [ ]:
with open('manifest.json', 'w') as fh:
    fh.write(content)